Importación de librerías.

In [1]:
import pandas as pd
import re
import gc
import spacy
import nltk
import sklearn
import wordcloud
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np
import json
from sklearn.model_selection import train_test_split


Descarga de los datos.

In [2]:
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/aysen.csv
nltk.download('stopwords')
nltk.download('wordnet')
!python -m spacy download es_core_news_sm
import es_core_news_sm
nlp_es = es_core_news_sm.load()

--2021-12-04 23:38:16--  https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/aysen.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29482924 (28M) [text/plain]
Saving to: ‘aysen.csv’

aysen.csv           100%[===================>]  28.12M  --.-KB/s    in 0.1s    

2021-12-04 23:38:18 (216 MB/s) - ‘aysen.csv’ saved [29482924/29482924]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 16.2 MB 8.2 MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-py3-none-any.whl size=16172933 sha256=c02199ffc10555373a4b7e8a498afc1f

# 1. Limpieza de la base de datos

Este capítulo del trabajo consiste en la limpieza de la base de datos de la lista de espera de la Región de Aysén. Se parte utilizando sólo las columnas de sospecha diagnóstica y la prestación a la que se deriva, para luego seleccionar las que tienen una cantidad mayor a 10.000 derivaciones y realizar un preprocesamiento del texto. 

## Carga base de datos Aysén

En primer lugar, se inspecciona la base de datos cargada y luego se seleccionan sólo las columnas "SOSPECHA_DIAG" y "PRESTA_EST". Estas columnas son ingresadas a la variable 'aysen_subset'. 



In [3]:
aysen = pd.read_csv("aysen.csv", sep=";", na_values=["Otro",""])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Selección de las columnas "SOSPECHA_DIAG" (sospecha diagnóstica) y "PRESTA_EST" (prestación).

In [4]:
aysen_subset = aysen[["SOSPECHA_DIAG","PRESTA_EST"]]

## 1.1 Derivaciones mayores a 10.000 - D10K 

En el presente apartado se cuentas las derivaciones a cada especialidad y se realiza un ránking con ellas.

Se calcula el dataframe D10K que contiene las prestaciones con más de 10.000 valores. Las especialidades se muestran de forma individual. 

In [5]:
D10K_rank = pd.DataFrame(aysen_subset.PRESTA_EST.value_counts(dropna=False))
D10K_rank = D10K_rank.loc[D10K_rank["PRESTA_EST"] > 10000]
D10K_rank = D10K_rank.reset_index()
D10K_rank.columns = ["PRESTA_EST", "PRESTA_EST_COUNT"]
D10K_rank


,PRESTA_EST,PRESTA_EST_COUNT
0,TRAUMATOLOGIA,24004
1,OFTALMOLOGIA,21482
2,CIRUGIA ADULTO,17321
3,OTORRINOLARINGOLOGIA,13663
4,NEUROLOGIA,12316
5,MEDICINA INTERNA,11408
6,GINECOLOGIA,10871
7,ENDODONCIA,10225


In [6]:
list(D10K_rank.PRESTA_EST)

['TRAUMATOLOGIA',
 'OFTALMOLOGIA',
 'CIRUGIA ADULTO',
 'OTORRINOLARINGOLOGIA',
 'NEUROLOGIA',
 'MEDICINA INTERNA',
 'GINECOLOGIA',
 'ENDODONCIA']

In [7]:
D10K = pd.merge(aysen_subset, D10K_rank["PRESTA_EST"], on="PRESTA_EST", how='right')

## 1.2 Normalización del texto

A continuación, se construyen las expresiones regulares necesarias para:
- Eliminar signos de puntuación.
- Cambiar todo el texto a minúsculas.
- Cambiar cualquier símbolo no-ASCII.

In [8]:
pattern = r"""(?x)                   # set flag to allow verbose regexps
              (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A.
              |\$?\d+(?:[.,]\d+)?%?  # numbers, incl. currency and percentages
              |\w+(?:[-']\w+)*       # words w/ optional internal hyphens/apostrophe
              |(?:[+/\-@&*¡!.,])     # special characters with meanings
            """

def normalizer(text, remove_tildes = True): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = nltk.regexp_tokenize(text, pattern)
    text = ' '.join(text)
    if remove_tildes:
        text = re.sub('á', 'a', text) #replaces special vowels to their base forms
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [9]:
D10K['SOSPECHA_DIAG_NORM'] = D10K.apply(lambda x: normalizer(x.SOSPECHA_DIAG), axis=1)

## 1.3. Stopwords

<font color='red'>**No entendí muy bien por qué se hizo esta unión de palabras que no están en común en la lista. Yo lo cambiaré y simplemente utilizaré simplemente la unión de las dos listas. +1 de Alvaro.**</font>

En primer lugar, se escoge la lista de stopwords a utilizar. En este caso se eligen dos listas de stopwords: nltk y spacy. Se importan las dos listas de stopwords y luego se realiza una unión de los dos conjutos de palabras. 

In [10]:
nltk_stopwords = nltk.corpus.stopwords.words('spanish')
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS

stop_words = set(list(nltk_stopwords) + list(spacy_stopwords))
stop_words = list(stop_words)

El conjunto de stopwords debe ser normalizado de la misma forma en que se normalizó el texto de la sospecha diagnóstica. Luego de esta normalización se eliminan los stopwords de la columna "SOSPECHA_DIAG_NORM" en D10K. 

In [11]:
stopwords_normalized = [normalizer(word) for word in stop_words]

def no_stopwords(text):
  '''
  Función para eliminar las stopwords de un texto
  '''
  text = text.split(" ")
  text_nosw = []
  for w in range(len(text)):
    if text[w] not in stopwords_normalized:
      text_nosw.append(text[w])
  separator = ' '
  text = separator.join(text_nosw)
  return text

La función descrita anteriormente permite eliminar los stopwords de un texto normalizado. Se presenta un ejemplo de esto:

Se aplica la función para quitar las stopwords a todas las filas de "SOSPECHA_DIAG_NORM" y se guardan en una nueva columna llamada "SOSPECHA_DIAG_NOSTOPWORDS".

# 3. Análisis supervisado

## 3.1 Probabilidad de cada categoría en D10K

Para calcular la probabilidad de cada categoría en D10K, primero obtendremos el total de categorías para luego dividir la cuenta de categoría por el total. Asignaremos una nueva columna en el dataframe para la probabilidad.

In [12]:
total_cat = D10K_rank.PRESTA_EST_COUNT.sum()
D10K_rank['PROB'] = D10K_rank.PRESTA_EST_COUNT/total_cat
D10K_rank

,PRESTA_EST,PRESTA_EST_COUNT,PROB
0,TRAUMATOLOGIA,24004,0.197906
1,OFTALMOLOGIA,21482,0.177113
2,CIRUGIA ADULTO,17321,0.142806
3,OTORRINOLARINGOLOGIA,13663,0.112647
4,NEUROLOGIA,12316,0.101542
5,MEDICINA INTERNA,11408,0.094056
6,GINECOLOGIA,10871,0.089628
7,ENDODONCIA,10225,0.084302


## 3.2 Diccionario P(palabra|categoria).

En esta sección se construirá un diccionario, donde para cada palabra se obtendrá la probabilidad P(palabra|categoria).
[IMAGEN]

Primero dividiremos la data de D10K en entrenamiento y test (90% y 10% respectivamente)

In [13]:
train, test = train_test_split(D10K, test_size=0.1)

A continuación, se construye el vector en base a los datos de entrenamiento quitando los stop_words

In [14]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(stop_words=stop_words)
term_doc_matrix_train = count_vectorizer.fit_transform(train.SOSPECHA_DIAG)
term_doc_matrix_train

<109161x4274 sparse matrix of type '<class 'numpy.int64'>'
	with 303353 stored elements in Compressed Sparse Row format>

Luego, se calcula la probabilidad de cada palabra por categoría

In [15]:
p_w_category = {}
p_w_s_category = {}
for index, row in D10K_rank.iterrows():
  category = row.PRESTA_EST
  count_c = np.asarray(term_doc_matrix_train.todense()[train.PRESTA_EST == category,:].sum(0)).reshape(-1).sum()
  # Se obtienen las apariciones de cada una de las palabras en cada clase
  c_w = np.asarray(term_doc_matrix_train.todense()[train.PRESTA_EST == category,:].sum(0)).reshape(-1)
  # Se calcula la probabilidad normal
  p_w = c_w / count_c
  # Se calcula la probabilidad suavizada para bayes
  vocab = len(np.asarray(term_doc_matrix_train.todense()[train.PRESTA_EST == category,:].sum(0)).reshape(-1))
  p_w_smoothing = (c_w + 1) / (count_c + vocab)
  
  p_w_category[category] = dict(zip(count_vectorizer.get_feature_names_out(), p_w))
  p_w_s_category[category] = dict(zip(count_vectorizer.get_feature_names_out(), p_w_smoothing))

## 3.3 *Construya un clasificador Bayesiano Naive. Reporte el resultado de 3 métricas de clasificación ¿Detecto alguna categoría de mayor dificultad? ¿puede sugerir alguna interpretación a su resultado?*


In [28]:
c_w_negative = np.asarray(term_doc_matrix_train.todense()[train.PRESTA_EST != 'TRAUMATOLOGIA', :].sum(0)).reshape(-1)
c_w_positive = np.asarray(term_doc_matrix_train.todense()[train.PRESTA_EST == 'TRAUMATOLOGIA', :].sum(0)).reshape(-1)
c_negative = (train.PRESTA_EST != 'TRAUMATOLOGIA').sum()
c_positive = (train.PRESTA_EST == 'TRAUMATOLOGIA').sum()
p_w_negative = (c_w_negative + 1) / c_negative # Probabilidad asociada a cada palabra en la clase negativa
p_w_positive = (c_w_positive + 1) / c_positive
r = np.log(p_w_positive / p_w_negative)
term_doc_matrix_test = count_vectorizer.transform(test.SOSPECHA_DIAG)
test_pred = term_doc_matrix_test.dot(r) > 0
test_label = np.where(test.PRESTA_EST == 'TRAUMATOLOGIA', True, False)
confusion_matrix = pd.crosstab(test_label,test_pred)
TP = confusion_matrix[1][1]
FP = confusion_matrix[1][0]
FN = confusion_matrix[0][1]
accuracy = (np.array(confusion_matrix).diagonal().sum() / np.array(confusion_matrix).sum())
precision = TP/(TP+FP)
recall = TP/(TP+FN)

In [30]:
print(f"accuracy={accuracy}, precision={precision}, recall={recall}")

accuracy=0.854151207848957, precision=0.6105436573311367, recall=0.7594262295081967


In [ ]:
for name, values in train.iteritems():
  if name not in ['PRESTA_EST', 'SOSPECHA_DIAG']:
    train.drop(name, axis=1, inplace=True)

for name, values in test.iteritems():
  if name not in ['PRESTA_EST', 'SOSPECHA_DIAG']:
    test.drop(name, axis=1, inplace=True)

In [32]:
def binary_naive_bayes(count_vectorizer, train_matrix, specialty):
  c_w_negative = np.asarray(train_matrix.todense()[train.PRESTA_EST != specialty, :].sum(0)).reshape(-1)
  c_w_positive = np.asarray(train_matrix.todense()[train.PRESTA_EST == specialty, :].sum(0)).reshape(-1)
  c_negative = (train.PRESTA_EST != specialty).sum()
  c_positive = (train.PRESTA_EST == specialty).sum()
  p_w_negative = (c_w_negative + 1) / c_negative # Probabilidad asociada a cada palabra en la clase negativa
  p_w_positive = (c_w_positive + 1) / c_positive
  r = np.log(p_w_positive / p_w_negative)
  term_doc_matrix_test = count_vectorizer.transform(test.SOSPECHA_DIAG)
  test_pred = term_doc_matrix_test.dot(r) > 0
  test_label = np.where(test.PRESTA_EST == specialty, True, False)
  confusion_matrix = pd.crosstab(test_label,test_pred)
  TP = confusion_matrix[1][1]
  FP = confusion_matrix[1][0]
  FN = confusion_matrix[0][1]
  accuracy = (np.array(confusion_matrix).diagonal().sum() / np.array(confusion_matrix).sum())
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  return accuracy, precision, recall

In [34]:
for specialty in list(D10K_rank.PRESTA_EST):
  accuracy, precision, recall = binary_naive_bayes(count_vectorizer, term_doc_matrix_train, specialty)
  print(f"specialty: {specialty}: accuracy={accuracy}, precision={precision}, recall={recall}")

specialty: TRAUMATOLOGIA: accuracy=0.854151207848957, precision=0.6105436573311367, recall=0.7594262295081967
specialty: OFTALMOLOGIA: accuracy=0.9154917965207354, precision=0.7551963048498845, recall=0.7676056338028169
specialty: CIRUGIA ADULTO: accuracy=0.7186907411987797, precision=0.3257992262268377, recall=0.940623162845385
specialty: OTORRINOLARINGOLOGIA: accuracy=0.9147497732706736, precision=0.6055261610817166, recall=0.7394113424264178
specialty: NEUROLOGIA: accuracy=0.8341165800972875, precision=0.3522012578616352, recall=0.7848309975267931
specialty: MEDICINA INTERNA: accuracy=0.6840629895292275, precision=0.2216842105263158, recall=0.8863636363636364
specialty: GINECOLOGIA: accuracy=0.694698656113447, precision=0.224468309117709, recall=0.9815837937384899
specialty: ENDODONCIA: accuracy=0.9598482974688762, precision=0.7362848893166506, recall=0.7822085889570553


In [ ]:
count_vectorizer = None
term_doc_matrix_train = None
gc.collect()

190

## 3.4 *¿Cómo varía su resultado al utilizar bigramas?*



In [35]:
count_vectorizer_bigrams = sklearn.feature_extraction.text.CountVectorizer(stop_words=stop_words, ngram_range=(1, 2))
term_doc_matrix_train_bigrams = count_vectorizer_bigrams.fit_transform(train.SOSPECHA_DIAG)

In [36]:
for specialty in list(D10K_rank.PRESTA_EST):
  accuracy, precision, recall=binary_naive_bayes(count_vectorizer_bigrams, term_doc_matrix_train_bigrams, specialty)
  print(f"specialty (with bigrams): {specialty}: accuracy={accuracy}, precision={precision}, recall={recall}")

specialty (with bigrams): TRAUMATOLOGIA: accuracy=0.8507708797097865, precision=0.6007677543186181, recall=0.769672131147541
specialty (with bigrams): OFTALMOLOGIA: accuracy=0.9157391376040894, precision=0.7555350553505535, recall=0.7690140845070422
specialty (with bigrams): CIRUGIA ADULTO: accuracy=0.7203396817544727, precision=0.3275545584336121, recall=0.9441504997060552
specialty (with bigrams): OTORRINOLARINGOLOGIA: accuracy=0.9042790007420233, precision=0.5618336886993603, recall=0.756640344580043
specialty (with bigrams): NEUROLOGIA: accuracy=0.8269436886800231, precision=0.3416726233023588, recall=0.7881286067600989
specialty (with bigrams): MEDICINA INTERNA: accuracy=0.6732624288894385, precision=0.21770091556459817, recall=0.9006734006734006
specialty (with bigrams): GINECOLOGIA: accuracy=0.6892571522796603, precision=0.2217382285832815, recall=0.9843462246777164
specialty (with bigrams): ENDODONCIA: accuracy=0.9538296644405969, precision=0.6876122082585279, recall=0.78323108